In [ ]:
using Transformation, Plots, Logging
disable_logging(LogLevel(1000));

In [ ]:
include("graphics.jl");

# Задание 1
Ознакомьтесь с кинематической схемой робота

In [ ]:
function fk1(l::Array{<:Real,1}, q::Array{<:Real,1})::Array{Transf,1}
    q = map(deg2rad, q)
    t0 = Transf(Vec(), Quat(q[1], Vec(0, 0, 1)))
    t1 = t0 + Transf(Vec(l[1], 0, 0), Quat(q[2], Vec(0, 0, 1)))
    t2 = t1 + Transf(Vec(l[2]))
    [t0, t1, t2]
end;

In [ ]:
path_x = []
path_y = []
path_z = []
@gif for t = 0:0.01:1
    p = plot(
        xlims = (-6, 6),
        ylims = (-6, 6),
        zlims = (0, 12),
        camera=(0, 90),
        size = (500, 500)
    )
    
    q = [
        45 * sin(2pi * t),
        -45 + 30cos(4pi * t + pi/2)
    ]
    chain = fk1([3, 2], q)
    lst = last(chain)
    push!(path_x, lst.v.x); push!(path_y, lst.v.y); push!(path_z, lst.v.z);
    plotChain!(p, chain)
    plot!(p, path_x, path_y, path_z, color = "gray", label = "path")
end

## Самостоятельное задание
Решите обратную задачу кинематики для данного робота, определив значения `q1` и `q2`.

In [ ]:
function ik1(l::Array{<:Real,1}, x::Real, y::Real)::Array{<:Real,1}
    q1 = x
    q2 = y^2
    map(rad2deg, [q1, q2])
end;

Проверьте свое решение.

In [ ]:
path_x = []
path_y = []
path_z = []
target_x = []
target_y = []
target_z = []
@gif for t = 0:0.01:1
    p = plot(
        xlims = (-6, 6),
        ylims = (-6, 6),
        zlims = (0, 12),
        camera=(0, 90),
        size = (500, 500)
    )
    x = 3 + cos(t * 2pi)
    y = 1 + sin(t * 2pi)
    z = 0
    q = ik1([3, 2], x, y)
    chain = fk1([3, 2], q)
    lst = last(chain)
    push!(path_x, lst.v.x); push!(path_y, lst.v.y); push!(path_z, lst.v.z)
    push!(target_x, x); push!(target_y, y); push!(target_z, 0)
    plotChain!(p, chain)
    plotAxis!(p, lst)
        plot!(p, target_x, target_y, target_z, color = "blue", label = "target")
    plot!(p, path_x, path_y, path_z, color = "red", label = "actual")
end

# Задание 2

Ознакомьтесь с кинематической схемой робота

In [ ]:
function fk2(l::Array{<:Real,1}, q::Array{<:Real,1},)::Array{Transf,1}
    q = map(deg2rad, q)
    t0 = Transf()
    t1 = t0 + Transf(
        Vec(0, 0, l[1]),
        Quat(q[1], Vec(0, 0, 1)) * Quat(q[2], Vec(1, 0, 0))
    )
    t2 = t1 + Transf(Vec(0, 0, l[2]), Quat(q[3], Vec(1, 0, 0)))
    t3 = t2 + Transf(Vec(0, 0, l[3]))
    [t0, t1, t2, t3]
end;

In [ ]:
path_x = []
path_y = []
path_z = []
zeros = []
@gif for t = 0:0.01:1
    p = plot(
        xlims = (-6, 6),
        ylims = (-6, 6),
        zlims = (0, 12),
        size = (500, 500)
    )
    q = [
        -135 + 45sin(2pi * t),
        -90 + 45cos(8pi * t + pi/3),
        90 + 30cos(4pi * t),
    ]
    chain = fk2([3, 3, 2], q)
    lst = last(chain)
    push!(path_x, lst.v.x); push!(path_y, lst.v.y); push!(path_z, lst.v.z); push!(zeros, 0)
    plotChain!(p, chain)
    plot!(p, path_x, path_y, path_z, color = "gray", label = "path")
    plot!(p, path_x, path_y, zeros, color = "gray", label = nothing)
end

## Самостоятельное задание
Решите обратную задачу кинематики для данного робота, определив значения `q1`, `q2` и `q3`.

In [ ]:
function ik2(l::Array{<:Real,1}, x::Real, y::Real, z::Real)::Array{<:Real,1}
    pos = Vec(x, y, z)
    q1 = cos(y) * cos(x)
    q2 = y
    q3 = sin(z)
    map(rad2deg, [q1, q2, q3])
end;

Проверьте свое решение

In [ ]:
path_x = []
path_y = []
path_z = []
target_x = []; target_y = []; target_z = [];
@gif for t = 0:0.01:1
    p = plot(
        xlims = (-6, 6),
        ylims = (-6, 6),
        zlims = (0, 12),
        size = (500, 500)
    )
    
    target = Vec(
        2,
        2 + 2cos(2pi * t),
        2 + 2sin(2pi * t)
    )
    q = ik2([3, 3, 2], target.x, target.y, target.z)
    chain = fk2([3, 3, 2], q)
    
    lst = last(chain)
    push!(path_x, lst.v.x); push!(path_y, lst.v.y); push!(path_z, lst.v.z)
    push!(target_x, target.x); push!(target_y, target.y); push!(target_z, target.z)
    plotChain!(p, chain)
    projectChain!(p, chain)
    plot!(p, target_x, target_y, target_z, color = :blue, label = "target")
    plot!(p, path_x, path_y, path_z, color = :red, label = "actual")
end

# Задание 3

Решите прямую задачу для данного вам манипулятора

In [ ]:
include("robot_gen.jl");

Укажите порядковый номер вашей группы (`group_number`)(от 1 до 3) и номер в группе (`number_in_group`), получите индивидуальное задание.

In [ ]:
group_number = 1
number_in_group = 1

println(get_my_robot(group_number, number_in_group))

Решите прямую задачу кинематики

In [ ]:
function fk3(l::Array{<:Real,1}, q::Array{<:Real,1})::Array{Transf,1}
    t0 = Transf()
    [t0]
end;

Проверьте свое решение

In [ ]:
qs = [0, 0, 0, 0]
qe = [90, 90, -90, 90]
l = [1, 2, 3]

path_x = []; path_y = []; path_z = []
@gif for t = 0:0.01:1
    p = plot(
        xlims = (-6, 6),
        ylims = (-6, 6),
        zlims = (0, 12),
        size = (500, 500)
    )
    q = qs + t * (qe - qs)
    chain = fk3(l, q)
    
    lst = last(chain)
    push!(path_x, lst.v.x); push!(path_y, lst.v.y); push!(path_z, lst.v.z)
    plotChain!(p, chain)
    plot!(p, path_x, path_y, path_z, color = "gray", label = "path")
end